# Quantization

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/salut

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/salut


In [3]:
!BUILD_CUDA_EXT=0 pip install -q auto-gptq transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are in

In [4]:
!pip install transformers -U
!pip install tokenizers==0.19.0
!pip install accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.4 MB/s eta 0:00:00


In [5]:
import random
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

base_id='gpt2-large'
model_id = "mddokmak/MNLP"

In [7]:
model = AutoModelForCausalLM.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/640 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/94.4M [00:00<?, ?B/s]

In [8]:
quantize_config_4 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
)

quantize_config_8 = BitsAndBytesConfig(
    load_in_8bit=True,
)

model_4bit = AutoModelForCausalLM.from_pretrained(base_id, quantization_config=quantize_config_4, torch_dtype='auto')
model_8bit = AutoModelForCausalLM.from_pretrained(base_id, quantization_config=quantize_config_8, torch_dtype='auto')
quantized_model_4bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantize_config_4, torch_dtype='auto')
quantized_model_8bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantize_config_8, torch_dtype='auto')
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

In [9]:
#compute the model size
def model_size(model):
    m_size = 0
    for param in model.parameters():
        if param.data.is_floating_point():
            m_size += param.numel() * torch.finfo(param.data.dtype).bits
        else:
            m_size += param.numel() * torch.iinfo(param.data.dtype).bits
    print(f"model size: {m_size} / bit | {m_size / 8e6:.2f} / MB | {m_size / 8e9:.2f} / GB ")
model_size(model)
model_size(model_8bit)
model_size(model_4bit)
model_size(quantized_model_8bit)
model_size(quantized_model_4bit)

model size: 25523937280 / bit | 3190.49 / MB | 3.19 / GB 
model size: 7782031360 / bit | 972.75 / MB | 0.97 / GB 
model size: 4950876160 / bit | 618.86 / MB | 0.62 / GB 
model size: 8537006080 / bit | 1067.13 / MB | 1.07 / GB 
model size: 5705850880 / bit | 713.23 / MB | 0.71 / GB 


In [10]:
model_8bit.save_pretrained("./quantized_base_model_8")
model_4bit.save_pretrained("./quantized_base_model_4")
quantized_model_8bit.save_pretrained("./quantized_model_8")
quantized_model_4bit.save_pretrained("./quantized_model_4")

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:399: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
